In [1]:
import pandas as pd

In [2]:
import pandas as pd
import os

dataset_names = ['yaz']  

processed_dfs = []

# Schleife durch jeden Datensatz und verarbeite ihn
for name in dataset_names:

    cv_drf_scores_path = f'/root/WorkingFolder/results/cv_drf_scores_{name}.csv'
    cv_scores_levelset_models_path = f'/root/WorkingFolder/results/cv_scores_levelset_models_{name}.csv'
    
    df_0 = pd.read_csv(cv_drf_scores_path)
    df = pd.read_csv(cv_scores_levelset_models_path)
    

    # Reshape 
    reshaped_df = df.melt(
        id_vars=['binSize', 'weightsByDistance', 'fold', 'model_name', 'cu', 'co', 'variable', 'dataset_name'],
        value_vars=['0.9', '0.75', '0.5', '0.25', '0.1'],
        var_name='tau',
        value_name='split_test_score'
    )
    
    # Pivot-Umformung für ein breites Format mit split0_test_score bis split4_test_score
    wide_df = reshaped_df.pivot_table(
        index=['binSize', 'weightsByDistance', 'model_name', 'cu', 'co', 'variable', 'tau'],
        columns='fold',
        values='split_test_score'
    ).reset_index()
    
    
    wide_df.columns = [
        'binSize', 'weightsByDistance', 'model_name', 'cu', 'co', 'variable', 'tau', 
        'split0_test_score', 'split1_test_score', 'split2_test_score', 
        'split3_test_score', 'split4_test_score'
    ]
    
    # Kombiniere 'wide_df' und 'df_0' mit einem outer join
    combined_df = pd.concat([wide_df, df_0], join='outer', ignore_index=True)
    

    columns_to_drop = [
        'mean_score_time', 'std_score_time', 'param_min_node_size', 
        'param_num_features', 'param_num_trees', 'mean_test_score',
        'std_test_score', 'rank_test_score', "mean_fit_time", "std_fit_time"
    ]
    combined_df = combined_df.drop(columns=columns_to_drop, errors='ignore')
    

    combined_df['dataset'] = name
    
    # Füge den verarbeiteten DataFrame zur Liste hinzu
    processed_dfs.append(combined_df)

# Alle verarbeiteten DataFrames zu einem einzigen DataFrame zusammenfügen
final_combined_df = pd.concat(processed_dfs, ignore_index=True) if processed_dfs else pd.DataFrame()

display(final_combined_df)


,binSize,weightsByDistance,model_name,cu,co,variable,tau,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,params,dataset
0,20.0,False,LS_KDEx_LGBM,1.0,9.0,calamari,0.1,0.940000,1.292683,1.380531,0.862069,1.612613,NaN,yaz
1,20.0,False,LS_KDEx_LGBM,1.0,9.0,chicken,0.1,0.971993,1.116340,1.041558,1.464220,1.007030,NaN,yaz
2,20.0,False,LS_KDEx_LGBM,1.0,9.0,fish,0.1,1.005319,0.945205,0.987879,0.875000,1.170068,NaN,yaz
3,20.0,False,LS_KDEx_LGBM,1.0,9.0,koefte,0.1,1.087059,0.865019,1.145000,0.573441,1.003824,NaN,yaz
4,20.0,False,LS_KDEx_LGBM,1.0,9.0,lamb,0.1,0.821510,0.824584,0.806265,0.715962,0.851899,NaN,yaz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,NaN,NaN,DRF,1.0,9.0,steak,0.1,-8.745493,-10.330465,-8.877747,-12.691905,-13.289343,"OrderedDict([('min_node_size', 64), ('num_feat...",yaz
2166,NaN,NaN,DRF,1.0,9.0,steak,0.1,-8.040187,-9.602100,-8.937015,-12.082886,-13.258602,"OrderedDict([('min_node_size', 64), ('num_feat...",yaz
2167,NaN,NaN,DRF,1.0,9.0,steak,0.1,-8.214593,-9.713175,-8.991307,-12.033247,-12.548300,"OrderedDict([('min_node_size', 32), ('num_feat...",yaz
2168,NaN,NaN,DRF,1.0,9.0,steak,0.1,-8.535985,-10.396170,-8.965021,-12.756089,-13.435056,"OrderedDict([('min_node_size', 64), ('num_feat...",yaz


In [5]:
import pandas as pd

# Definiere die Liste der Datensatznamen
dataset_names = ['yaz', 'air', 'wage']  # Füge weitere Datensatznamen bei Bedarf hinzu

# Initialisiere eine leere Liste, um die einzelnen DataFrames zu speichern
combined_dfs = []

# Schleife durch jeden Datensatz und verarbeite ihn
for name in dataset_names:
    # Erzeuge den Dateipfad, indem 'yaz' durch den aktuellen Datensatznamen ersetzt wird
    file_path = f'/root/WorkingFolder/results/results_basic_Models_{name}.csv'
    
    # Lese die CSV-Datei
    df = pd.read_csv(file_path)
    
    # Füge eine neue Spalte mit dem Datensatznamen hinzu
    df['dataset'] = name
    
    # Füge den verarbeiteten DataFrame zur Liste hinzu
    combined_dfs.append(df)

# Alle verarbeiteten DataFrames zu einem einzigen DataFrame zusammenfügen
final_combined_df = pd.concat(combined_dfs, ignore_index=True)

# Optional: Zeige die letzten 5 Zeilen des finalen kombinierten DataFrames an
display(final_combined_df)


,Variable,cu,co,Model,Pinball Loss,Best Params,delta C,sl,dataset
0,calamari,9.0,1.0,SAA,0.511842,NaN,NaN,0.9,yaz
1,calamari,9.0,1.0,MLP,0.494512,"OrderedDict([('alpha', 0.001), ('early_stoppin...",0.033858,0.9,yaz
2,calamari,9.0,1.0,LGBM,0.389481,"OrderedDict([('learning_rate', 0.01), ('max_de...",0.239060,0.9,yaz
3,calamari,9.0,1.0,RFW,0.473684,"OrderedDict([('max_depth', 8), ('max_features'...",0.074550,0.9,yaz
4,calamari,9.0,1.0,KNNW,0.375000,"OrderedDict([('n_neighbors', 32)])",0.267352,0.9,yaz
...,...,...,...,...,...,...,...,...,...
1710,22,1.0,9.0,LGBM,0.200487,"OrderedDict([('learning_rate', 0.05), ('max_de...",-0.811759,0.1,wage
1711,22,1.0,9.0,RFW,0.095530,"OrderedDict([('max_depth', 10), ('max_features...",0.136716,0.1,wage
1712,22,1.0,9.0,KNNW,0.092257,"OrderedDict([('n_neighbors', 32)])",0.166297,0.1,wage
1713,22,1.0,9.0,DTW,0.110895,"OrderedDict([('max_depth', 2), ('min_samples_s...",-0.002130,0.1,wage
